<a href="https://colab.research.google.com/github/nguyen-nhat-mai/object_detection/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model setup and initial installations

In [ ]:
from google.colab import drive
drive = drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Installing the yolov5 setup
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import os
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-165-ge382d4c Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


Setup complete ✅ (12 CPUs, 83.5 GB RAM, 23.4/166.8 GB disk)


In [ ]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.0.0+cu118 (NVIDIA A100-SXM4-40GB)


Link of the general documentation: https://docs.ultralytics.com/yolov5/

#Download and preparation of the dataset
We need a dataset organized as follow:

  -3 folders train, val and test

  -In each of these folder, a folder labels and a folder images

  The hyp.custom.airportscanner.yaml file contains hyperparameters (such as learning rate, losses parameters, etc.) that can be modified for the training.

  The dataset.airportscanner.yaml file contains information about labels and paths for the dataset.

  Note that this notebook have been configured to be used on GoogleColab and that the model is based on Pytorch.

In [ ]:
#Copy the dataset to the good location
newpath = '/content/yolov5/datasets/airportscanner' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

#/content/drive/MyDrive/CRPSharedFolder/CRP__Dataset_Repartition = the path of your dataset
%cp -R /content/drive/MyDrive/CRP_Shared_Folder/CRP__Dataset_Repartition/Scenario_14 /content/yolov5/datasets/airportscanner 
%cp -R /content/drive/MyDrive/CRP_Shared_Folder/CRP__Dataset_Repartition/valid /content/yolov5/datasets/airportscanner 
%cp -R /content/drive/MyDrive/CRP_Shared_Folder/CRP__Dataset_Repartition/test /content/yolov5/datasets/airportscanner 

In [ ]:
# Change the folder name to train if needed
os.rename('/content/yolov5/datasets/airportscanner/Scenario_14', '/content/yolov5/datasets/airportscanner/train')

In [ ]:
#Download the custom hyperparameter files (you can change the hyperparameters by modifying these files)
!wget -P /content/yolov5/datasets/airportscanner https://raw.githubusercontent.com/nguyen-nhat-mai/object_detection/main/data/hyp.custom.airportscanner.finetuning.yaml
!wget -P /content/yolov5/datasets/airportscanner https://raw.githubusercontent.com/nguyen-nhat-mai/object_detection/main/data/hyp.custom.airportscanner.headtraining.yaml
#Download the information file regarding the dataset. It describes the classes as well as the train/val/test paths
!wget -P /content/yolov5/datasets/airportscanner https://raw.githubusercontent.com/nguyen-nhat-mai/object_detection/model_maxime/dataset.airportscanner.yaml

--2023-05-09 15:32:11--  https://raw.githubusercontent.com/nguyen-nhat-mai/object_detection/model_maxime/hyp.custom.airportscanner.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1407 (1.4K) [text/plain]
Saving to: ‘/content/yolov5/datasets/airportscanner/hyp.custom.airportscanner.yaml’

hyp.custom.airports 100%[===================>]   1.37K  --.-KB/s    in 0s      

2023-05-09 15:32:11 (39.1 MB/s) - ‘/content/yolov5/datasets/airportscanner/hyp.custom.airportscanner.yaml’ saved [1407/1407]

--2023-05-09 15:32:11--  https://raw.githubusercontent.com/nguyen-nhat-mai/object_detection/model_maxime/dataset.airportscanner.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting t

#Training

Do not forgot to save your best model after training!
Note that, plots, predicted images examples and other can be found in the runs/train folder

In [ ]:
%cd /content/yolov5 #Ensure you are in the good location

In [ ]:
#Initial training of the head of the network (we freeze the 12 first layers)
#You can personnalize your hyperparameter file (hyp.custom.airportscanner.yaml)
!python train.py \
--batch 32 \
--epochs 100 \
--data /content/yolov5/datasets/airportscanner/dataset.airportscanner.yaml \
--weights /content/yolov5/yolov5m6_airportIoU05/initial/weights/best.pt\
--optimizer Adam \
--cache \
--hyp /content/yolov5/datasets/airportscanner/hyp.custom.airportscanner.headtraining.yaml\
--project 'yolov5m6_airportscanner'\
--name 'FrozenTrain'\ 
--freeze 12 

In [ ]:
#Finetuning of the model. We unfreeze the whole network and take a small learning rate
#Note that you need to modify the parameters (the learning rate at least) in the hyp.custom.airportscanner.yaml file
!python train.py \
--batch 32 \
--epochs 100 \
--data /content/yolov5/datasets/airportscanner/dataset.airportscanner.yaml \
--weights /content/yolov5/yolov5m6_airportIoU05/initial/weights/best.pt\
--optimizer Adam \
--cache \
--hyp /content/yolov5/datasets/airportscanner/hyp.custom.airportscanner.finetuning.yaml\
--project 'yolov5m6_airportscanner'\
--name 'Finetuning'


#Training monitoring

In [ ]:
#To visualize the training on tensorboard dev
#logdir = precise the path to the result of the run
#name = name of your experiment
#description = your description
#one_shot = means that it stops the workflow once everything is uploaded


In [ ]:
!tensorboard dev upload --logdir runs/train \
  --name "YOLOv5m6" \
  --description "Scenario_14_IoU_theshold03" \
  --one_shot


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

runs/train

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

To sign in with the TensorBoard uploader:

1. On your computer or phone, visit:

   https://www.google.com/device

2. Sign in with your Google account, then enter:

   TTH-KFR-TQP



New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/HZQEaOR4Qh65u93OuMNC2A/

[2023-05-09T03:34:38] Started scanning logdir.
[2023-05-09T03:34:41] Total uploaded: 1664 scalars